In [13]:
import pandas as pd
import random
import string
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score

In [7]:
# Data loading
dirty_data = pd.read_csv('../../datasets/initial_datasets/dirty_titles.csv')
clean_data = pd.read_csv('../../datasets/cleaned_datasets/cleaned_titles.csv', on_bad_lines='skip')

/var/folders/29/hp9758mj5zj7dcbpxts76vt00000gn/T/ipykernel_79532/3781380141.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  dirty_data = pd.read_csv('../../datasets/initial_datasets/dirty_titles.csv')


In [10]:
clean_data = pd.read_csv('../../datasets/cleaned_datasets/cleaned_titles.csv', on_bad_lines='skip')

In [8]:
# Funzione per generare una stringa casuale di 6 caratteri
def random_string(length=6):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def replace_null_or_n(value, column):
    if pd.isnull(value) or value == r"\N":
        if column == 'genres':
            return random_string()
        elif column in ['titleType', 'primaryTitle', 'originalTitle']:
            return 'null_value'
        elif column == 'isAdult':
            return 1
        elif column in ['startYear', 'endYear']:
            return 1900
        elif column == 'runtimeMinutes':
            return 1
        elif column == 'tconst': 
            return 'tttttttttt00'
    return value

In [9]:
# Applica la funzione a tutte le colonne del DataFrame
for column in dirty_data.columns:
    dirty_data[column] = dirty_data[column].apply(lambda x: replace_null_or_n(x, column))

# Mostra le ultime 5 righe del DataFrame
print(dirty_data.tail(5))

                tconst   titleType primaryTitle originalTitle isAdult  \
64027483  tttttttttt00  null_value   null_value    null_value       1   
64027484  tttttttttt00  null_value   null_value    null_value       1   
64027485  tttttttttt00  null_value   null_value    null_value       1   
64027486  tttttttttt00  null_value   null_value    null_value       1   
64027487  tttttttttt00  null_value   null_value    null_value       1   

         startYear endYear runtimeMinutes  genres  
64027483      1900    1900              1  bfpcnt  
64027484      1900    1900              1  pjabqx  
64027485      1900    1900              1  qozrll  
64027486      1900    1900              1  qjpslu  
64027487      1900    1900              1  vqjfbd  


In [11]:
'''# fields: tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres

# Selection of features variables (columns) and target variable (column)
features = ['titleType', 'primaryTitle', 'isAdult', 'runtimeMinutes'] 
target = 'genres'

# Creation of dataframes with features and target
X_dirty = dirty_data[features]
y_dirty = dirty_data[target]

X_clean = clean_data[features]
y_clean = clean_data[target]

# Split data for training and testing
X_dirty_train, X_dirty_test, y_dirty_train, y_dirty_test = train_test_split(X_dirty, y_dirty, test_size=0.2, random_state=42)
X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)
'''

In [14]:
# 2. Preprocessare i dati
# Codificare la colonna 'titleType' con LabelEncoder
label_encoder = LabelEncoder()
dirty_data['titleType'] = label_encoder.fit_transform(dirty_data['titleType'])

# Binarizzare la colonna 'genres'
dirty_data['genres'] = dirty_data['genres'].apply(lambda x: x.split(','))
mlb = MultiLabelBinarizer()
genres_binarized = mlb.fit_transform(dirty_data['genres'])

# Utilizzare TF-IDF per la colonna 'primaryTitle'
tfidf = TfidfVectorizer()
primaryTitle_tfidf = tfidf.fit_transform(dirty_data['primaryTitle'])

# Concatenare tutte le features
import numpy as np
X = np.hstack((dirty_data[['titleType', 'isAdult', 'runtimeMinutes']].values, primaryTitle_tfidf.toarray()))

# La colonna target è 'genres'
y = genres_binarized

# 3. Dividere il dataset in train e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Addestrare il modello
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 5. Valutare il modello
y_pred = model.predict(X_test)

# Calcolare la accuracy e f1 score per ogni etichetta di genere e poi fare una media
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')


MemoryError: Unable to allocate 18.5 PiB for an array with shape (64027488, 40693691) and data type int64